In [ ]:
import xarray as xr
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib import cm
from matplotlib import ticker
from matplotlib.lines import Line2D
import matplotlib.patches as patches

import sys, warnings, glob
warnings.filterwarnings("once", category=RuntimeWarning)

## The function below performs smoothing on power spectra in frequency space

In [ ]:
##############################################################
### Function to smooth out ripples in data in frequency space
def freq_smooth2(freq,psd,winfac=1):
    """
    Required inputs:
    freq   (float 1D array) = frequencies (ascending) [Hz]
    psd    (float 1D or 2D array) = corresponding power spectral density [v**2/Hz]
    
    Optional inputs:
    winfac          (float) = scaling factor for window size as function of frequency
    
    Output:
    smooth (float 1D or 2D array) = Smoothed version of psd
    """
    
    smooth = psd.copy()
    if freq.ndim != 1:
        raise IndexError("Frequencies must be in a 1D array")
    win = np.rint(np.exp(np.sqrt(freq)*winfac))*2 - 1 # Window width
    
    if len(psd.shape) == 1:  # 1D case - easy:  
        if freq.shape != psd.shape:
            raise IndexError("Required input series are not the same length")
        for n in range(len(freq)):
            t0 = int((win[n]-1)/2)
            t1 = np.min((int((win[n]-1)/2)+1,len(freq)))
            smooth[n] = psd[n-t0:n+t1].mean()
            
    elif len(psd.shape) == 2:  # 2D case - harder:
        if q_lmean.wavelength.size not in psd.shape:
            raise IndexError("No PSD array dimension matches frequency series length")
        idx = psd.shape.index(freq.size) # This is the matching dimension for frequencies
        for n in range(len(freq)):
            t0 = int((win[n]-1)/2)
            t1 = np.min((int((win[n]-1)/2)+1,len(freq)))
            for j in range(psd.shape[1-idx]):
                if idx == 1:
                    smooth[:,n] = psd[:,n-t0:n+t1].mean(axis=idx)   
                else:
                    smooth[n,:] = psd[n-t0:n+t1,:].mean(axis=idx)   
        
    else:
        raise IndexError("Required PSD array cannot exceed 2D")
        
    return smooth

### Functions for custom color maps

In [ ]:
##############################################################
### Gradient Maker code

import sys
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors


def hex_to_rgb(hex_string):
    '''
    Converts hex string to rgb colors
    
    Required input:
        hex_string (str) = String of characters representing a hex color.
                           May or ay not have a leading `#`
                           Any alpha value on the end will be stripped off
    Output:
        A list (length 3) of integers (range 0-255) of RGB values
    '''
    value = hex_string.strip("#")[:6] # removes hash symbol if present, alpha value on end
    lv = len(value)
    return tuple(int(value[i:i + lv // 3], 16) for i in range(0, lv, lv // 3))


def rgb_to_hex(tup3):
    '''
    Converts rgb 3-member list or array to hex colors
    
    Required input: 
        tup3 (tuple) = list (length 3) of RGB values in decimal 0-255 range   
        
    Output: 
        A String of 6 characters representing a hex color, with leading hash.
    '''
    return "#{:02x}{:02x}{:02x}".format(tup3[0],tup3[1],tup3[2])


def rgb_to_dec(rgb_list):
    '''
    Converts RGB (range 0-255) to decimal (range 0-1) colors (i.e. divides each value by 255)
    
    Required input:
        rgb_list (list): list (length 3) of RGB values (range 0-255)
        
    Output:
        A list (length 3) of decimal values (0-1)
    '''
    return [v/255 for v in rgb_list[:3]]


def dec_to_rgb(dec_list):
    '''
    Converts decimal (range 0-1) to RGB (range 0-255) colors
    
    Required input:
        dec_list (list): list (length 3) of decimal values (0-1)
        
    Output:
        A list (length 3) of RGB values (range 0-255)
    '''
    return [int(round(v*255)) for v in dec_list[:3]]


def get_continuous_cmap(hex_list, float_list=None, ncol=256):
    ''' 
    Creates and returns a color map that can be used in heat map figures.
        The parameter `float_list` can be used to control the spacing between colors:
            If float_list is not provided, color map graduates linearly between each color in hex_list.
            If float_list is provided, each color in hex_list is mapped to the respective relative location in float_list. 
        
    Required input:
        hex_list   (list): List of hex code strings
        
    Optional input:
        float_list (list): List of floats between 0 and 1, same length as hex_list. Must start with 0 and end with 1.
        ncol        (int): Number of colors in the final colormap - default is 256
    
    Output:
        A color map in matplotlib `LinearSegmentedColormap` object 
        '''
    
    rgb_list = [rgb_to_dec(hex_to_rgb(i)) for i in hex_list]
    if float_list:
        pass
    else:
        float_list = list(np.linspace(0,1,len(rgb_list)))
        
    cdict = dict()
    for num, col in enumerate(['red', 'green', 'blue']):
        col_list = [[float_list[i], rgb_list[i][num], rgb_list[i][num]] for i in range(len(float_list))]
        cdict[col] = col_list
    cmp = mcolors.LinearSegmentedColormap('my_cmp', segmentdata=cdict, N=ncol)
    return cmp



def grad_brite(xrgb,pcol=None,lite_0=0.0,lite_1=1.0,mid_lite=None,mid_spot=None, ncol=256):
    '''
    Remaps the color sequence xrgb (required) into a constant gradient of brightness 
        on the greyscale in 1 or 2 linear segments.
    Result is a color scale to use as a cmap in plotting.
    
    Required input: 
        xrgb     (list) = a list of rgb hex strings (form: 'Xrrggbb') giving sequence of colors;
                          it is assumed the first color at a position of 0.0 and the last at 1.0 for 
                          the interpolations described below.
    
    Optional arguments:
        pcol      (list) = a list of float positions on a scale 0.0-1.0 corresponding to colors in xrgb;
                           must be same length as xrgb (default is equally spaced)
                
        lite_0   (float) = brightness on scale 0.0-1.0 for first color in list (default 0.0)
        
        lite_1   (float) = brightness on scale 0.0-1.0 for last color in list (default 1.0)
        
        mid_lite (float) = if present, defines an intermediate point between 0.0 and 1.0 where
                           a third brightness level is defined. Two linear gradients of brightness 
                           are determined - one from 0.0 to mid_lite and one from mid_lite to 1.0.
                           Otherwise, a single linear gradient from positions 0.0 to 1.0 is defined.
        
        mid_spot (float) = position between 0.0 and 1.0 where mid_lite is placed;
                           ignored if mid_lite=None
                
        ncol       (int) = number of colors in the final colormap (default is 256)
        
    Returned results: 
        cmp     (object) = a ncol-stepped colormap object to use as a cmap in plotting
        
        cmp_r   (object) = The reversed version of cmp
    '''

    luma = [0.30, 0.59, 0.11]      # NTSC luma perception weightings for R, G, B
        
    # Nip potential problems with inputs
    lite_0 = np.max([0.0,np.min([1.0,lite_0])]) # Range is 0-1
    lite_1 = np.min([1.0,np.max([0.0,lite_1])]) # Range is 0-1
    ncol   = np.max([2,np.min([1024,ncol])])    # Range is 2-1024
    
    # Produce a preliminary color map, size ncol, from input colors - brightness will be adjusted below
    if pcol:
        if pcol[0] != 0.0:
            sys.exit("ERROR - First color position must be 0.0.")
        if pcol[-1] != 1.0:
            sys.exit("ERROR - Last color position must be 1.0.")
        if len(xrgb) != len(pcol):
            sys.exit("ERROR - List of positions is not the same length as list of colors.")
        else:
            rez = get_continuous_cmap(xrgb, float_list=pcol, ncol=ncol) # Produce color gradient - linear interp'd
    else:
        rez = get_continuous_cmap(xrgb, ncol=ncol) # Produce color gradient - default to equally spaced

    # Find the brightnesses of the originally supplied/requested colormap
    lite_rez = [np.dot(luma,rez(i)[:3]) for i in range(rez.N)]
        
    # Map out the target brightesses for the color map based on inputs
    if mid_lite:         # Two linear segments to interpolate
        if mid_spot:
            mid_x = int(np.rint(mid_spot*(ncol-1)))
            mid_x = np.max([np.min([mid_x,(ncol-2)]),1]) # Ensures midpoint is not same as first or last point
            lite = [lite_0 + (mid_lite-lite_0)*i/mid_x for i in range(mid_x)] + [mid_lite + (lite_1-mid_lite)*i/(ncol-mid_x-1) for i in range(ncol-mid_x)]
        else:
            sys.exit("ERROR - Location for intermediate brightness unspecified.")
    else:                # Only one segment
        lite = [lite_0 + (lite_1-lite_0)*i/(ncol-1) for i in range(ncol)] 

    # Rescaling of brightnesses at each point to produce perceptually uniform gradients
    fact = np.array(lite)/np.array(lite_rez)    
    fact3 = np.repeat(fact, repeats=3).reshape(ncol,3)   # Expand out to 3 array elements for array multiplication
    nurez3 = np.multiply(fact3,rez(range(ncol))[:,:3])   # Scaled values into a numpy array
    nurez3 = np.where(nurez3>1.0,1.0,np.where(nurez3<0.0,0.0,nurez3))
    nurez4 = np.ones((ncol,4)) ; nurez4[:,:-1] = nurez3  # Tack the alphas back on (all set to opaque)
    
    # Create a matplotlib colormap list for output
    cmp = mcolors.ListedColormap(nurez4)
    cmp_r = mcolors.ListedColormap(np.flip(nurez4,axis=0))
    
    return cmp,cmp_r


### Some large arrays and strings are set below to be used to annotate the dataset produced below

In [ ]:
# Approximate pressures [Pa] corresponding to model levels (just a time average from one of the cases)
p_levs = np.array([97741.44 , 97398.18 , 97057.56 , 96719.125, 96382.766, 96048.21 ,
       95715.01 , 95382.93 , 95051.836, 94721.61 , 94392.29 , 94063.93 ,
       93736.61 , 93410.234, 93084.7  , 92760.07 , 92436.27 , 92113.336,
       91791.305, 91470.1  , 91149.69 , 90830.11 , 90511.375, 90193.49 ,
       89876.516, 89560.45 , 89245.33 , 88931.15 , 88617.95 , 88305.68 ,
       87994.25 , 87683.74 , 87374.19 , 87065.664, 86758.15 , 86451.63 ,
       86146.15 , 85841.67 , 85538.15 , 85235.555, 84933.87 , 84633.02 ,
       84332.984, 84033.75 , 83735.375, 83437.89 , 83141.336, 82845.664,
       82550.92 , 82257.12 , 81964.19 , 81672.19 , 81381.08 , 81090.89 ,
       80801.55 , 80513.   , 80225.305, 79938.43 , 79652.38 , 79367.18 ,
       79082.86 , 78799.44 , 78516.85 , 78235.08 , 77954.195, 77674.195,
       77395.02 , 77116.67 , 76839.12 , 76562.3  , 76286.25 , 76010.984,
       75736.52 , 75462.9  , 75190.05 , 74917.96 , 74646.664, 74376.14 ,
       74106.49 , 73837.71 , 73569.74 , 73302.67 , 73036.484, 72771.305,
       72507.17 , 72244.055, 71981.96 , 71720.85 , 71460.63 , 71201.58 ,
       70943.81 , 70687.1  , 70431.24 , 70176.24 , 69922.11 , 69669.07 ,
       69417.2  , 69166.414, 68916.56 , 68667.766, 68419.945, 68172.92 ,
       67926.734, 67681.36 , 67436.77 , 67192.9  , 66949.695, 66707.21 ,
       66465.414, 66224.28 , 65983.88 , 65744.23 , 65505.37 , 65267.23 ,
       65029.78 , 64793.008, 64556.918, 64321.508, 64086.785, 63852.684,
       63619.258, 63386.53 , 63154.508, 62923.2  , 62692.56 , 62462.605,
       62233.395, 62004.88 , 61777.07 , 61549.95 , 61323.496, 61097.684,
       60872.566, 60648.1  , 60424.316, 60201.258, 59978.914, 59757.258,
       59536.215, 59315.805, 59095.945, 58876.695, 58658.   , 58439.887,
       58222.355, 58005.477, 57789.383, 57574.125, 57359.594, 57145.78 ,
       56932.754, 56720.55 , 56509.07 , 56298.285, 56088.215, 55878.82 ,
       55670.117, 55462.133, 55254.883, 55048.38 , 54842.582, 54637.402,
       54432.78 , 54228.758, 54025.383, 53822.62 , 53620.414, 53413.402,
       53195.96 , 52967.26 , 52727.133, 52475.32 , 52210.676, 51933.094,
       51642.125, 51336.582, 51016.254, 50680.44 , 50328.12 , 49958.906,
       49571.812, 49166.34 , 48741.938, 48297.59 , 47833.152, 47347.863,
       46840.688, 46310.96 , 45757.793, 45180.402, 44578.145, 43949.965,
       43294.848, 42612.16 , 41901.027, 41161.133, 40391.66 , 39591.812,
       38761.625, 37902.418, 37015.66 , 36099.28 , 35151.277, 34172.605,
       33164.355, 32126.564, 31060.488, 29966.203, 28843.734, 27694.73 ,
       26527.67 , 25374.379, 24258.316, 23178.521, 22135.934, 21129.643,
       20157.207, 19218.822, 18314.855, 17444.225, 16605.197, 15797.964,
       15021.669, 14274.116, 13556.278, 13073.933])
# Approximate heights [m AGL] corresponding to model levels (just a time average from one of the cases)
z_levs = np.array([   15.697305,    47.003498,    78.152885,   109.18822 ,
         140.12819 ,   170.99742 ,   201.83646 ,   232.66306 ,
         263.48798 ,   294.31656 ,   325.14484 ,   355.9643  ,
         386.77017 ,   417.56982 ,   448.3696  ,   479.1682  ,
         509.96713 ,   540.7652  ,   571.5618  ,   602.35944 ,
         633.1591  ,   663.9615  ,   694.76434 ,   725.56433 ,
         756.3584  ,   787.1455  ,   817.9245  ,   848.6918  ,
         879.4479  ,   910.1955  ,   940.9394  ,   971.6786  ,
        1002.4047  ,  1033.1143  ,  1063.8088  ,  1094.4874  ,
        1125.1477  ,  1155.7919  ,  1186.4244  ,  1217.0476  ,
        1247.6613  ,  1278.2714  ,  1308.8835  ,  1339.496   ,
        1370.1034  ,  1400.703   ,  1431.293   ,  1461.8727  ,
        1492.4412  ,  1522.9967  ,  1553.543   ,  1584.0798  ,
        1614.6058  ,  1645.1223  ,  1675.6332  ,  1706.143   ,
        1736.652   ,  1767.1573  ,  1797.6582  ,  1828.1549  ,
        1858.6423  ,  1889.1199  ,  1919.5938  ,  1950.0632  ,
        1980.5247  ,  2010.9766  ,  2041.4221  ,  2071.8645  ,
        2102.3044  ,  2132.7468  ,  2163.1902  ,  2193.6335  ,
        2224.0728  ,  2254.5073  ,  2284.9392  ,  2315.3723  ,
        2345.806   ,  2376.2366  ,  2406.659   ,  2437.0747  ,
        2467.486   ,  2497.892   ,  2528.2888  ,  2558.6653  ,
        2589.0173  ,  2619.3489  ,  2649.6565  ,  2679.9517  ,
        2710.24    ,  2740.493   ,  2770.6968  ,  2800.8716  ,
        2831.0376  ,  2861.2036  ,  2891.366   ,  2921.5012  ,
        2951.5933  ,  2981.662   ,  3011.7202  ,  3041.7522  ,
        3071.766   ,  3101.7798  ,  3131.7915  ,  3161.7935  ,
        3191.7917  ,  3221.795   ,  3251.802   ,  3281.812   ,
        3311.8276  ,  3341.8484  ,  3371.869   ,  3401.884   ,
        3431.8918  ,  3461.895   ,  3491.8982  ,  3521.9026  ,
        3551.9094  ,  3581.9119  ,  3611.916   ,  3641.9243  ,
        3671.9336  ,  3701.9375  ,  3731.9363  ,  3761.933   ,
        3791.9285  ,  3821.9216  ,  3851.9084  ,  3881.8914  ,
        3911.8716  ,  3941.8499  ,  3971.8286  ,  4001.8088  ,
        4031.7869  ,  4061.764   ,  4091.7395  ,  4121.7046  ,
        4151.6597  ,  4181.609   ,  4211.556   ,  4241.5034  ,
        4271.4565  ,  4301.4126  ,  4331.3726  ,  4361.3364  ,
        4391.305   ,  4421.273   ,  4451.221   ,  4481.1436  ,
        4511.056   ,  4540.9644  ,  4570.859   ,  4600.734   ,
        4630.601   ,  4660.466   ,  4690.3296  ,  4720.19    ,
        4750.0503  ,  4779.906   ,  4809.75    ,  4839.5845  ,
        4869.413   ,  4899.247   ,  4929.0938  ,  4958.9478  ,
        4988.8003  ,  5018.6523  ,  5048.5146  ,  5079.1836  ,
        5111.4976  ,  5145.5986  ,  5181.529   ,  5219.3506  ,
        5259.2583  ,  5301.2915  ,  5345.5483  ,  5392.237   ,
        5441.4194  ,  5493.2476  ,  5547.9185  ,  5605.5356  ,
        5666.307   ,  5730.3726  ,  5797.89    ,  5869.096   ,
        5944.101   ,  6023.112   ,  6106.4023  ,  6194.1836  ,
        6286.7466  ,  6384.3604  ,  6487.284   ,  6595.8438  ,
        6710.4106  ,  6831.275   ,  6958.7915  ,  7093.227   ,
        7234.981   ,  7384.4844  ,  7542.1406  ,  7708.2354  ,
        7882.8984  ,  8066.8276  ,  8260.93    ,  8465.5625  ,
        8681.12    ,  8908.353   ,  9147.682   ,  9399.853   ,
        9665.779   ,  9946.119   , 10239.759   , 10539.291   ,
       10838.875   , 11138.76    , 11438.572   , 11738.272   ,
       12038.477   , 12339.073   , 12639.703   , 12940.302   ,
       13241.187   , 13542.021   , 13842.673   , 14143.909   ,
       14445.741   , 14724.665   ])

####################################################
# Monotonic color scale light to dark, reddish scale 
x_toasty = ['#FFFFFF', '#FFF6C0', '#ef3018', '#6C2610', '#6C2610']
p_toasty = [0.0, 0.073, 0.30, 0.6, 1.0] 
lite_0=0.99; lite_1=0.10; mid_lite = 0.5; mid_spot=0.5  
lite_0=1.00; lite_1=0.0; mid_lite = 1.00; mid_spot=0.073 
ncol = 256
toasty,toasty_r = grad_brite(x_toasty,pcol=p_toasty,lite_0=lite_0,lite_1=lite_1,mid_lite=mid_lite,mid_spot=mid_spot,ncol=ncol)
toasty

In [ ]:
#################################
# Open files

ddir = "/Volumes/SSD_8TB/CLASP/LES_runs2/" # Path to output from `spatial_PSD_data.ipynb`

# Open files
d00 = xr.open_dataset(f"{ddir}fr2_PSD_00.nc4")
d01 = xr.open_dataset(f"{ddir}fr2_PSD_01.nc4")

d00

In [ ]:
v_sfc = ['PSD_SH','PSD_LH'] # These are now in pairs by index
v_atm = ['PSD_TH','PSD_QV']
l_sfc = ["Surface Sensible Heat Flux","Surface Latent Heat Flux"]
l_atm = ["Atmospheric Potential Temperature","Atmospheric Specific Humidity"]
l_panel = ['a','b']

frequency = 1/(d00.wavelength.data*250)

# Setting up the plots
fig,ax = plt.subplots(1,2,figsize=(10,6),sharey='all')
plt.subplots_adjust(wspace=0.02,hspace=0.02,left=0.08,right=0.90,bottom=0.09,top=0.88)  

# Loop through variable pairs
for j,va in enumerate(v_atm):
    vs = v_sfc[j]
    # Find the total power for these variables across domain each time step
    a_HET = d00[va].assign_coords({"p_levs":('bottom_top',p_levs), 
                                   "z_levs":('bottom_top',z_levs),
                                   "frequency":('wavelength',frequency)}).integrate('frequency')
    a_HOM = d01[va].assign_coords({"p_levs":('bottom_top',p_levs), 
                                   "z_levs":('bottom_top',z_levs),
                                   "frequency":('wavelength',frequency)}).integrate('frequency')
    s_HET = d00[vs].assign_coords({"frequency":('wavelength',frequency)}).integrate('frequency')

    # Rearrange the time axis by date (case) vs hour and for each hour, correlate across cases
    s_ratio = s_HET.coarsen(time=14).construct(time=("case","hour")).assign_coords({"lt_hour":('hour',range(7,21))})
    a_ratio = (a_HET/a_HOM).coarsen(time=14).construct(time=("case","hour")).assign_coords({"lt_hour":('hour',range(7,21))})
    corr = xr.corr(np.log(s_ratio),np.log(a_ratio),dim='case')
        
    ax[j].pcolormesh(corr.lt_hour,corr.p_levs/100,(corr*corr).T.where(abs(corr.T)>0.27),vmin=0,vmax=0.6,cmap=toasty)
    ax[j].yaxis.set_inverted(True) 
    #ax[j].yaxis.tick_right()
    ax[j].set_xticks(range(7,21,2))
    ax[j].xaxis.set_major_formatter(ticker.StrMethodFormatter("{x:02}"))
    if j == 0:
        ax[j].yaxis.set_label_position("left")
        ax[j].set_ylabel(f"Pressure [hPa]",fontsize=14)
        
    ax[j].set_title(f"{l_atm[j]}\nvs\n{l_sfc[j]}",fontsize=14)
    ax[j].set_xlabel(f"Local Hour",fontsize=14)
    ax[j].tick_params(axis='both',labelsize=13)
    
    box_props = dict(facecolor='#f0f5fa',edgecolor='#e0e0e0',alpha=0.5)
    # place a text box in upper left in axes coords
    ax[j].text(6.8,144,f"{l_panel[j]})",va='top',ha='left',bbox=box_props,fontsize=16)
# Color bar
cbax1 = fig.add_axes([0.927, 0.35, 0.02, 0.40]) 
cbar1 = fig.colorbar(cm.ScalarMappable(cmap=toasty),cax=cbax1,extend='neither',orientation='vertical')
cbar1.ax.tick_params(labelsize=13)

# new clear axis overlay with 0-1 limits
axp = plt.axes([0,0,1,1], facecolor=(1,1,1,0))
#xl,yl = np.array([[0.927,0.947], [0.3792,0.3792]])
#line = Line2D(xl,yl,lw=0.5,color='k')
#axp.add_line(line)
rect = patches.Rectangle((0.927,0.35),0.020,0.0292,linewidth=0.75,edgecolor='k',facecolor='w')
axp.add_patch(rect)
axp.set_axis_off()
        
fig.savefig("heterogeneity_correlation_plot.pdf")